In [ ]:
%load_ext autoreload
%autoreload 2
import os
import itertools
import json
import shap
import sys
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from collections import defaultdict
from scipy import stats
from sqlalchemy import create_engine

sys.path.insert(0, os.path.abspath(".."))
from qtrees.helper import get_logger, init_db_args
from qtrees.constants import NOWCAST_FEATURES, FORECAST_FEATURES, HYPER_PARAMETERS_NC, HYPER_PARAMETERS_FC
from qtrees.data_processor import DataLoader, PreprocessorForecast, PreprocessorNowcast
sys.path.insert(0, os.path.abspath('../../qtrees-ai-data-private/'))
from qtreesprivate.plots_util import apply_qtrees_style

style.use("../../qtrees-ai-data-private/data/baumblick.mplstyle")
pd.set_option('display.max_columns', None)
DEPTH_MAP = {1: "30 cm", 2: "60 cm", 3: "90 cm"}

np.random.seed(42)

In [ ]:
def pick_test_weeks(weeks, n_train_weeks=5, n_test_weeks=4):
    train_weeks, test_weeks = [], []

    i = 0
    while i + n_train_weeks  < len(weeks):
        # Define the training period
        train_start = weeks[i]
        train_end = weeks[i + min(n_train_weeks, len(weeks) - i) - 1]

        # Define the testing period
        test_start = weeks[i + n_train_weeks]
        if i + n_train_weeks + n_test_weeks < len(weeks):
            test_end = weeks[i + n_train_weeks + n_test_weeks -1]
        else:
            test_end = weeks[-1]
        
        train_weeks.extend(range(train_start, train_end + 1))  
        test_weeks.extend(range(test_start, test_end + 1)) 

        i = i + n_train_weeks + n_test_weeks 

    return train_weeks, test_weeks

def create_train_test_split_across_sites(data, n_splits=4):
    train_data_folds, test_data_folds = [], []

    # Create a KFold object
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    site_ids = data.site_id.unique()

    for train_index, test_index in kf.split(site_ids):
        train_data = data[data.site_id.isin(site_ids[train_index])]
        test_data = data[data.site_id.isin(site_ids[test_index])]

        # calculate the feature across only the train data and use it also for testing
        temp = train_data.groupby(["timestamp","type_id"])["value"].mean().shift(1).rename("mean_yesterday")
        train_data = train_data.merge(temp, left_on=["timestamp", "type_id"], right_index=True)
        test_data = test_data.merge(temp, left_on=["timestamp", "type_id"], right_index=True)

        train_data_folds.append(train_data.dropna())
        test_data_folds.append(test_data.dropna())

    return list(zip(train_data_folds, test_data_folds))


def log_experiment_results(fold_results, experiment_id="experiment1", model="RandomForestRegressor", features=[], hyper_parameters={}, csv_file="experiments.csv"):
    df = pd.DataFrame([{"experiment_id": experiment_id, 
                        "Mean 30cm": json.dumps({key: np.round(np.mean([d[key] for d in fold_results["Folds 30cm"]]),2) for key in fold_results["Folds 30cm"][0]}),
                        "Mean 60cm": json.dumps({key: np.round(np.mean([d[key] for d in fold_results["Folds 60cm"]]),2) for key in fold_results["Folds 60cm"][0]}),
                        "Mean 90cm": json.dumps({key: np.round(np.mean([d[key] for d in fold_results["Folds 90cm"]]),2) for key in fold_results["Folds 90cm"][0]}),
                        "Folds 30cm": json.dumps(fold_results["Folds 30cm"]), 
                        "Folds 60cm": json.dumps(fold_results["Folds 60cm"]), 
                        "Folds 90cm": json.dumps(fold_results["Folds 90cm"]), 
                        "model": model, "features": json.dumps(features), 
                        "hyper_parameters": json.dumps(hyper_parameters)}])

    # Check if the file already exists; if not, write the header
    write_header = not pd.io.common.file_exists(csv_file)

    with open(csv_file, 'a', newline='') as file:
        df.to_csv(file, mode='a', index=False, header=write_header)


def read_experiment_log(csv_file):
    data = pd.read_csv(csv_file)
    data['hyper_parameters'] = data['hyper_parameters'].apply(json.loads)
    data['features'] = data['features'].apply(json.loads)
    return data

def predict_depth(model, test_data, type_id = 1, features=[]):
    X_test, y_test = test_data.loc[test_data.type_id==type_id, features], test_data.loc[test_data.type_id==type_id,["value"]]
    
    y_hat =  model.predict(X_test) # y_hat = np.expm1(model.predict(X_test))
    rmse = mean_squared_error(y_test, y_hat, squared=False)
    mae = mean_absolute_error(y_test, y_hat)

    y_test.rename(columns={"value": "y_test"})
    y_test["y_hat"] = y_hat
    y_test["tree_id"] = test_data.loc[test_data.type_id==type_id, "tree_id"]
    y_test["timestamp"] = test_data.loc[test_data.type_id==type_id, "timestamp"]
    y_test["benchmark"] = test_data.loc[test_data.type_id==type_id, "mean_yesterday"]

    #print(f"Random Forest: Tiefe {DEPTH_MAP[type_id]}: RMSE {rmse:.2f}, MAE {mae:.2f}")
    return {"rmse": round(rmse, 2), "mae": round(mae, 2)}, y_test


def predict_benchmark(test_data, type_id = 1, features=[]):
    _, y_test = test_data.loc[test_data.type_id==type_id, features], test_data.loc[test_data.type_id==type_id, "value"]
    y_hat_benchmark = test_data.loc[test_data.type_id==type_id, "mean_yesterday"]
    rmse = mean_squared_error(y_test, y_hat_benchmark, squared=False)
    mae = mean_absolute_error(y_test, y_hat_benchmark)
    return {"rmse": round(rmse, 2), "mae": round(mae, 2)}, y_hat_benchmark

def evaluate_benchmark_on_folds(folds, log_experiment=True):
    fold_results = defaultdict(list)
    for fold in folds:
        fold_results["Folds 30cm"].append(predict_benchmark(fold[1], type_id=1)[0])
        fold_results["Folds 60cm"].append(predict_benchmark(fold[1], type_id=2)[0])
        fold_results["Folds 90cm"].append(predict_benchmark(fold[1], type_id=3)[0])
    if log_experiment:
        log_experiment_results(fold_results, experiment_id="Benchmark", model="Benchmark", features=[], hyper_parameters={})
        
    return fold_results

def init_result_folder(name):
    folder_path = os.path.join(".", name)  # Use "." to create the folder in the current directory

    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
    else:
        os.makedirs(folder_path)

    return folder_path

def plot_predictions(predictions, path):
    y_min, y_max = 0.0, predictions.value.max()
    min_date, max_date = predictions.timestamp.min(), predictions.timestamp.max()
    idx = pd.DatetimeIndex(pd.date_range(start=min_date, end=max_date, freq='D'))

    for tree in predictions.tree_id.unique():
        plot_data = predictions[(predictions.tree_id==tree) & (predictions.type_id==1)].set_index("timestamp").sort_index()
        
        if len(plot_data) > 0:
            mae_forecast = mean_absolute_error(plot_data["value"], plot_data["y_hat"])
            mae_benchmark = mean_absolute_error(plot_data["value"], plot_data["benchmark"])
            plot_data = plot_data.reindex(idx)
            
            fig, ax = plt.subplots(1,2, figsize=(12,5))
            plot_data.value.plot(ax=ax[0], label="Sensorwert")
            plot_data.y_hat.plot(ax=ax[0], label="Vorhersage")
            plot_data.benchmark.plot(ax=ax[0], label="Benchmark")
            ax[0].set_ylim((y_min, y_max))
            ax[0].set_xlim(("2022-04", "2022-09"))
            ax[0].xaxis.grid(False)

            plot_data.value.plot(ax=ax[1], label="Sensorwert")
            plot_data.y_hat.plot(ax=ax[1], label="Vorhersage")
            plot_data.benchmark.plot(ax=ax[1], label="Benchmark")
            ax[1].set_ylim((y_min, y_max))
            ax[1].set_xlim(("2023-04", "2023-09"))
            #ax[1].legend()
            ax[1].xaxis.grid(False)

            apply_qtrees_style(fig,f"Tree {tree}, Tiefe 30cm",f"Vorhersage MAE: {mae_forecast:.2f}, Benchmark MAE: {mae_benchmark:.2f}")
            ax[0].get_legend().set_visible(False)
        
        plt.savefig(os.path.join(path, f"forecast_{tree}.png"), bbox_inches="tight")

def evaluate_folds(folds, name, model, hyper_parameters, features, explain_model=False, log_experiment=True):
    fold_results = defaultdict(list)
    predictions = []
    path = init_result_folder(name)
    
    for idx, fold in enumerate(folds):
        prediction_list = []
        X_train, y_train = fold[0][features], fold[0]["value"]
        for depth in [1, 2, 3]:
            model.fit(X_train, y_train)
            results, pred = predict_depth(model, fold[1], type_id=depth, features=features)
            fold_results[f"Folds {30*depth}cm"].append(results)
            pred["type_id"] = depth
            prediction_list.append(pred)

        fold_predictions = pd.concat(prediction_list, ignore_index=True)
        fold_predictions["fold"] = idx
        predictions.append(fold_predictions)
    
        if explain_model and idx == 0:
            feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
            feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
            feature_importance_df.to_csv(os.path.join(path, f"feature_importance_fold_{idx}.csv"))
            feature_importance_df.set_index("Feature").plot(kind="barh")
            plt.savefig(os.path.join(path, f"feature_importance_fold_{idx}.png"), bbox_inches="tight")
            
            mask = np.random.choice(range(len(X_train)),size=200, replace=False)
            explainer = shap.Explainer(model, X_train.iloc[mask,])
            shap_values = explainer(X_train)
            shap.plots.bar(shap_values)
            plt.savefig(os.path.join(path, f"shap_values_{idx}.png"), bbox_inches="tight")

            shap.plots.violin(shap_values)
            plt.savefig(os.path.join(path, f"shap_values_violin_{idx}.png"), bbox_inches="tight")


    if log_experiment:
        log_experiment_results(fold_results, experiment_id=name, model=model.__class__.__name__, features=features, hyper_parameters=hyper_parameters)
        
    total_predictions = pd.concat(predictions, ignore_index=True)
    total_predictions.to_csv(os.path.join(path, f"predictions_fold_{idx}.csv"))

    #plot_predictions(total_predictions, path)

    return fold_results, total_predictions

def results_dict_to_df(res_dict, name):
    x = pd.DataFrame({"rmse":0,"mae":0, "name":name}, index=["Folds 30cm", "Folds 60cm", "Folds 90cm"])
    for ind in x.index:
        for metric in ["rmse","mae"]:
            temp = 0
            for i in range(len(res_dict)):
                temp += res_dict[ind][i][metric]
            x.loc[ind,metric] = temp/len(res_dict)
    return x

# Preprocessed data (DEV)

In [ ]:
db_qtrees, postgres_passwd = "",""#Add user and password
engine = create_engine(
    f"postgresql://qtrees_user:{postgres_passwd}@{db_qtrees}:5432/qtrees"
    )

# Eval run nowcast

In [ ]:
enc = DataLoader(engine) #Downloads data
train_data_raw = enc.download_training_data(forecast=False)
prep_nowcast = PreprocessorNowcast()
prep_nowcast.fit(train_data_raw)
train_data_nowcast = prep_nowcast.transform_train(train_data_raw)
train_data_nowcast.rename(columns={"target":"value"},inplace=True)
train_data_nowcast.reset_index(inplace=True)

In [ ]:
### Things to change
#for md in [2, 3, 4, 5, 8, 13]:

#hyper_parameters=dict(n_estimators=1000, learning_rate=0.1, max_depth=5, random_state=42)
#model = XGBRegressor(**hyper_parameters)
data_eval = train_data_nowcast.drop(columns="mean_yesterday")
folds = create_train_test_split_across_sites(data_eval, n_splits=4)

#This is already optimised. I did the grid search
hyper_parameter_grid = [
    dict(n_estimators=200, learning_rate=0.01, max_depth=5, max_features='sqrt'),
    dict(fit_intercept=True),
    dict(max_features="sqrt", criterion="squared_error", n_estimators=300, max_depth=5, bootstrap=True)
]
models = [
    GradientBoostingRegressor,
    LinearRegression,
    RandomForestRegressor
]
res_list = []
pred_list = []
for hyper_parameters, model in zip(hyper_parameter_grid,models):
    results, predictions = evaluate_folds(folds, model.__name__, model(**hyper_parameters), hyper_parameters=hyper_parameters, features=NOWCAST_FEATURES, explain_model=False)
    res_list.append(results_dict_to_df(results, model.__name__))
    predictions["model"] = model.__name__
    pred_list.append(predictions)
#results = evaluate_benchmark_on_folds(folds, log_experiment=True)

In [ ]:
temp = pred_list[0].copy()
temp["model"] = "benchmark"
temp = temp[["value","benchmark","tree_id","type_id","model"]]
temp = temp.rename(columns={"benchmark":"y_hat"})

pred_results = pd.concat([pd.concat(pred_list)[["value","y_hat","tree_id","type_id","model"]],temp])
pred_results["error"] = abs(pred_results["y_hat"] - pred_results["value"])

pred_results["type_id"] = pd.Categorical(pred_results["type_id"]).rename_categories(list(DEPTH_MAP.values()))
pred_results = pred_results.rename(columns={"type_id":"Sensortiefe"})

In [ ]:
fig, ax = plt.subplots()
ax = sns.boxplot(data =pred_results, x="Sensortiefe", y="error", hue="model")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
apply_qtrees_style(fig,"Violinenplots der absoluten Fehler","für drei optimierte Modelle und Benchmark")

For Shap values no cross validation. Just train on the entire dataset and look at the results per depth

In [ ]:
for type_id in [1]:
    model = RandomForestRegressor(**HYPER_PARAMETERS_NC)
    X_train = train_data_nowcast[train_data_nowcast.type_id == type_id][NOWCAST_FEATURES]
    y_train = train_data_nowcast[train_data_nowcast.type_id == type_id]["value"]
    na_mask = X_train.isna().sum(axis=1) == 0
    X_train, y_train = X_train[na_mask], y_train[na_mask]
    model.fit(X_train, y_train)
    mask = np.random.choice(range(len(X_train)),size=200, replace=False)
    explainer = shap.Explainer(model, X_train.iloc[mask,])
    shap_values = explainer(X_train)
    shap.plots.bar(shap_values)
    shap.plots.violin(shap_values)

In [ ]:
result_benchmark = evaluate_benchmark_on_folds(folds, log_experiment=True)
res_benchmark = results_dict_to_df(result_benchmark,"benchmark")

In [ ]:
res_smartgrp = pd.concat(res_list + [res_benchmark]).set_index(["name"], append=True).reorder_levels([1,0])
res_smartgrp

# Unfiltered data (PROD)

In [ ]:
db_qtrees, postgres_passwd = "",""#Add user and password
engine_prod = create_engine(
    f"postgresql://qtrees_user:{postgres_passwd}@{db_qtrees}:5432/qtrees"
    )

In [ ]:
enc_prod = DataLoader(engine_prod) #Downloads data
train_data_raw_prod = enc_prod.download_training_data(forecast=False)
prep_nowcast_prod = PreprocessorNowcast()
prep_nowcast_prod.fit(train_data_raw_prod)
train_data_prod = prep_nowcast_prod.transform_train(train_data_raw_prod)
train_data_prod.rename(columns={"target":"value"},inplace=True)
train_data_prod.reset_index(inplace=True)

## No grouping

In [ ]:
### Things to change
#for md in [2, 3, 4, 5, 8, 13]:
#hyper_parameters=dict(n_estimators=1000, learning_rate=0.1, max_depth=5, random_state=42)
#model = XGBRegressor(**hyper_parameters)
data_eval_prod = train_data_prod.drop(columns="mean_yesterday")
folds_prod = create_train_test_split_across_sites(data_eval_prod, n_splits=4)

hyper_parameter_grid = [
    dict(max_features="sqrt",criterion="squared_error", n_estimators=1000, max_depth=5, bootstrap=True),
    dict(n_estimators=100, learning_rate=0.03, max_depth=3, max_features="sqrt"),
    dict(fit_intercept=True)
]
models = [
    RandomForestRegressor,
    GradientBoostingRegressor,
    LinearRegression
]

res_list_prod = []
for hyper_parameters, model in zip(hyper_parameter_grid,models):
    results, predictions = evaluate_folds(folds_prod, "Prod run no group: " + model.__name__, model(**hyper_parameters), hyper_parameters=hyper_parameters, features=NOWCAST_FEATURES)
    res_list_prod.append(results_dict_to_df(results, model.__name__))
#results = evaluate_benchmark_on_folds(folds, log_experiment=True)

In [ ]:
result_benchmark = results_dict_to_df(evaluate_benchmark_on_folds(folds_prod, log_experiment=True),"benchmark")
res_prod_nogrp = pd.concat(res_list_prod + [result_benchmark]).set_index(["name"], append=True).reorder_levels([1,0])

In [ ]:
res_smartgrp / res_prod_nogrp

In [ ]:
res_prod_nogrp

Result: Preprocessing works really well. We can decrease error by A LOT. Made sense to put so much energy into it

## Naive grouping approach: Just average over the sites

In [ ]:
train_grp = train_data_prod.groupby(["site_id","type_id","timestamp"]).mean(numeric_only=True).dropna().reset_index()
train_grp["tree_id"] = train_grp["site_id"]

In [ ]:
data_eval_prod = train_grp.drop(columns="mean_yesterday")
folds_prod = create_train_test_split_across_sites(data_eval_prod, n_splits=4)

hyper_parameter_grid = [
    dict(max_features="sqrt",criterion="squared_error", n_estimators=300, max_depth=5, bootstrap=True),
    dict(n_estimators=100, learning_rate=0.03, max_depth=3, max_features="sqrt"),
    dict(fit_intercept=True)
]
models = [
    RandomForestRegressor,
    GradientBoostingRegressor,
    LinearRegression
]

res_list_prod = []
for hyper_parameters, model in zip(hyper_parameter_grid,models):
    results, predictions = evaluate_folds(folds_prod, "Prod run naive group: " + model.__name__, model(**hyper_parameters), hyper_parameters=hyper_parameters, features=NOWCAST_FEATURES)
    res_list_prod.append(results_dict_to_df(results, model.__name__))

In [ ]:
result_benchmark = results_dict_to_df(evaluate_benchmark_on_folds(folds_prod, log_experiment=True),"benchmark")
res_prod_naivegrp = pd.concat(res_list_prod + [result_benchmark]).set_index(["name"], append=True).reorder_levels([1,0])

In [ ]:
res_prod_naivegrp.mean()

Naive grouping does not really help at all

# Evaluation of the forecast method

In [ ]:
test_weeks = [pd.Timestamp(x, tz="UTC") for x in ["2022-06-16","2022-09-01","2023-05-01","2023-08-01"]]
drop_weeks = [pd.Timestamp(x, tz="UTC") for x in ["2022-06-09","2022-08-25","2023-04-24","2023-07-24"]]
test_window = pd.concat([pd.date_range(x,periods=14, tz="UTC").to_series() for x in test_weeks])
drop_window = pd.concat([pd.date_range(x,periods=21, tz="UTC").to_series() for x in drop_weeks])

In [ ]:
prep_fc = PreprocessorForecast()
prep_fc.fit(train_data_raw)
forecast_data_eval = prep_fc.transform_train(train_data_raw)
test_forecast = forecast_data_eval[forecast_data_eval.index.get_level_values(1).isin(test_window)]
train_forecast = forecast_data_eval[~forecast_data_eval.index.get_level_values(1).isin(drop_window)]

In [ ]:
FEATURES_MVP = FORECAST_FEATURES + ["shift_1","shift_2","shift_3"]

y_train_fc = train_forecast["target"]
X_train_fc = train_forecast.drop(columns="target")
mask = X_train_fc.isna().sum(axis=1) == 0
y_train_fc = y_train_fc[mask]
X_train_fc = X_train_fc[mask]
model = RandomForestRegressor(**HYPER_PARAMETERS_FC)
model.fit(X_train_fc[FEATURES_MVP],y_train_fc)

In [ ]:
test_forecast = test_forecast.dropna()
test_fc_noautoreg = test_forecast.drop(columns=["shift_1","shift_2","shift_3"])

In [ ]:
pred_results = pd.DataFrame(data = {'timestamp':test_forecast.index.get_level_values(1), 
                     'tree_id': test_forecast.index.get_level_values(0), 
                     'y_true': test_fc_noautoreg["target"], 
                     "y_pred":0})
pred_results.set_index(['timestamp', 'tree_id'], inplace=True)
autoreg_features = test_forecast.reset_index(level=0).loc[test_weeks][["type_id","tree_id","shift_1","shift_2","shift_3"]].reset_index()
pred_results = None
for h in range(14):
    dates = [x + dt.timedelta(days=h) for x in test_weeks]
    temp = test_fc_noautoreg.reset_index(level=0).loc[dates].set_index(["tree_id", "type_id"], append=True)
    temp = temp.merge(autoreg_features, how="inner", left_index=True, right_on = ["timestamp", "tree_id", "type_id"]).drop(columns="target").set_index(["tree_id","timestamp"])
    ind = temp.reset_index().set_index(["timestamp", "tree_id", "type_id"]).index
    res = model.predict(temp[model.feature_names_in_])
    if pred_results is None:
        pred_results = pd.DataFrame(data = {"y_pred":res},index=ind)
        pred_results["window"] = h+1
    else:
        temp = pd.DataFrame(data = {"y_pred":res},index=ind)
        temp["window"] = h+1
        pred_results = pd.concat([pred_results, temp])
    autoreg_features = autoreg_features.set_index(['timestamp','tree_id','type_id']).loc[ind].reset_index()
    autoreg_features["shift_3"] = autoreg_features["shift_2"]
    autoreg_features["shift_2"] = autoreg_features["shift_1"]
    autoreg_features["shift_1"] = res
    autoreg_features["timestamp"] = autoreg_features["timestamp"] + dt.timedelta(days=1)
pred_results = pred_results.merge(test_fc_noautoreg.reset_index().set_index(["timestamp","tree_id","type_id"]).loc[pred_results.index,"target"], left_index=True, right_index=True)

In [ ]:
print('MAE: ', mean_absolute_error(pred_results["target"], pred_results["y_pred"]), '    RMSE: ', mean_squared_error(pred_results["target"], pred_results["y_pred"], squared=False))

In [ ]:
pd.DataFrame(dict(importance=model.feature_importances_),index=model.feature_names_in_).sort_values("importance").plot.barh(legend=False)

In [ ]:
mask = np.random.choice([False,True],size=X_train_fc.shape[0], replace=True, p=[0.94,0.06])
explainer = shap.Explainer(model, X_train_fc.loc[mask, FEATURES_MVP])
shap_values = explainer(X_train_fc.loc[mask, FEATURES_MVP])
shap.plots.bar(shap_values)
shap.plots.violin(shap_values)

In [ ]:
shap_res = np.abs(shap_values.values)
pd.DataFrame(dict(shap_value=shap_res.sum(axis=0)/shap_res.sum()), index=shap_values.feature_names).sort_values("shap_value", ascending=False)[3:7]

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
ax.plot(pred_results.reset_index().groupby("window").apply(lambda x: np.sqrt(mean_squared_error(x['target'], x['y_pred']))))
ax.plot(pred_results.reset_index().groupby("window").apply(lambda x: mean_absolute_error(x['target'], x['y_pred'])))
apply_qtrees_style(fig,"Fehler der Vorhersage nach Prognosehorizont", "RMSE (grün) und MAE (orange)")
ax.set_xlabel("Prognosehorizont")

In [ ]:
pred_results["error"] = pred_results["target"] - pred_results["y_pred"]
sigma = np.std(pred_results["error"])
mu = np.mean(pred_results["error"])
x = np.linspace(-100, 100, 201)

pred_results["error"].hist(bins=20, density=True)
plt.plot(x, stats.norm.pdf(x, mu, sigma))

In [ ]:
trees = ["00008100:0024c2fe", "00008100:002927d6"]

fig, axs = plt.subplots(nrows = len(trees),figsize = (10,5))
for i, tree in enumerate(trees):
    plot_df = pred_results.loc[("2023-08",tree,1),("y_pred","target")]
    before = forecast_data_eval[forecast_data_eval.type_id==1].loc[tree].sort_index().loc[drop_weeks[3]:test_weeks[3]-dt.timedelta(days=1),"target"].rename("old")
    if before.shape[0] > 0:
        plot_df = plot_df.merge(before, how="outer",left_index=True,right_index=True)
        plot_df.loc["2023-07-31",["y_pred","target"]] = plot_df.loc["2023-07-31","old"]
    axs[i].plot(plot_df.sort_index())
    axs[i].axvline(dt.date(2023,8,1),color="grey")
    axs[i].set_xticks(["2023-07-24", "2023-07-31","2023-08-07", "2023-08-14"],labels=["24.07.23", "31.07.23","07.08.23", "14.08.23"])
fig.legend(["Vorhersage","Zielwert","Vor Evaluation"])
apply_qtrees_style(fig,"14-Tage-Vorhersage anhand zweier beispielhafter Bäume","für den Evaluationszeitraum Anfang August 2023 auf 30cm Tiefe")


In [ ]:
for tree_id in pred_results.index.get_level_values(1).unique():
    plot_df = pred_results.loc[("2023-08",tree_id,1),("y_pred","target")]
    if plot_df.shape[0] > 0:
        plot_df = plot_df.assign(tree_id=tree_id)
        before = forecast_data_eval[forecast_data_eval.type_id==1].loc[tree_id].sort_index().loc[drop_weeks[3]:test_weeks[3]-dt.timedelta(days=1),"target"].rename("old")
        if before.shape[0] > 0:
            plot_df = plot_df.merge(before, how="outer",left_index=True,right_index=True)
            plot_df.loc["2023-07-31",["y_pred","target"]] = plot_df.loc["2023-07-31","old"]
        plot_df.sort_index().plot()

# Forecasting a full season with starting values

Trying to understand whether we actually need sensors

In [ ]:
forecast_fullseason_data = forecast_data_eval.sort_index(level=1).reset_index(level=0).loc["2023"].set_index("tree_id",append=True).reorder_levels([1,0])
autoreg_features = forecast_fullseason_data.index.get_level_values(0).unique()
autoreg_features = autoreg_features.to_frame().merge(pd.DataFrame({"timestamp":pd.Timestamp("2023-04-01",tz="UTC"),"type_id":[1,2,3]}), how="cross")
autoreg_features = autoreg_features.assign(shift_1=20, shift_2=20, shift_3=20)
target = forecast_fullseason_data.set_index("type_id",append=True)["target"]
forecast_fullseason_data = forecast_fullseason_data.drop(columns=["shift_1","shift_2","shift_3"])

In [ ]:
train_season_forecast = forecast_data_eval.sort_index(level=1).reset_index(level=0).loc[:"01-01-2023"].set_index("tree_id",append=True).reorder_levels([1,0])

In [ ]:
FEATURES_MVP = FORECAST_FEATURES + ["shift_1","shift_2","shift_3"]

y_train_fc = train_season_forecast["target"]
X_train_fc = train_season_forecast.drop(columns="target")
X_train_fc = X_train_fc[FEATURES_MVP + ["type_id"]]
mask = X_train_fc.isna().sum(axis=1) == 0
y_train_fc = y_train_fc[mask]
X_train_fc = X_train_fc[mask]
model = RandomForestRegressor(**HYPER_PARAMETERS_FC)
model.fit(X_train_fc,y_train_fc)

In [ ]:
pred_results = None
date = dt.date(2023,4,1)
window = 0
while date < dt.date(2023, 10, 1):
    temp = forecast_fullseason_data.reset_index(level=0).sort_index().loc[date.strftime("%Y-%m-%d")].set_index(["tree_id", "type_id"], append=True)
    temp = temp.merge(autoreg_features, how="left", left_index=True, right_on = ["timestamp", "tree_id", "type_id"]).drop(columns="target").set_index(["tree_id","timestamp"])
    temp.loc[:,["shift_1","shift_2","shift_3"]] = temp.loc[:,["shift_1","shift_2","shift_3"]].fillna(20)
    temp = temp[model.feature_names_in_].dropna()
    ind = temp.reset_index().set_index(["timestamp", "tree_id", "type_id"]).index
    res = model.predict(temp)
    if pred_results is None:
        pred_results = pd.DataFrame(data = {"y_pred": res},index=ind)
    else:
        res_step = pd.DataFrame(data = {"y_pred": res},index=ind)
        pred_results = pd.concat([pred_results, res_step])
    autoreg_features = temp.set_index("type_id",append=True).loc[:,["shift_1","shift_2","shift_3"]].reset_index()
    autoreg_features["shift_3"] = autoreg_features["shift_2"]
    autoreg_features["shift_2"] = autoreg_features["shift_1"]
    autoreg_features["shift_1"] = res
    autoreg_features["timestamp"] = autoreg_features["timestamp"] + dt.timedelta(days=1)
    date = date + dt.timedelta(days=1)
    window += 1

In [ ]:
full_season_predictions = pred_results.reorder_levels([1,0,2]).merge(target, how="left", left_index=True, right_index=True)
full_season_predictions["error"] = full_season_predictions["target"] - full_season_predictions["y_pred"] 
print("MAE: " + str(full_season_predictions["error"].abs().mean()))
print("RMSE: " + str(full_season_predictions["error"].std()))
print("Bias: " + str(full_season_predictions["error"].mean()))

In [ ]:
for tree_id in full_season_predictions.index.get_level_values(0).unique():
    temp = full_season_predictions.loc[tree_id,:,1].loc[:,["y_pred","target"]]
    if temp.shape[0] > 0:
        temp.plot()

In [ ]:
mask = np.random.choice([False,True],size=X_train_fc.shape[0], replace=True, p=[0.96,0.04])
explainer = shap.Explainer(model, X_train_fc.loc[mask, :])
shap_values = explainer(X_train_fc.loc[mask, :])
shap.plots.bar(shap_values)
shap.plots.violin(shap_values)